Importing the necessery libraries to read and manipulate the data and to use the Decision Tree model

In [10]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import accuracy_score
import category_encoders as ce
from sklearn.model_selection import train_test_split
from io import StringIO
from IPython.display import Image  
import pydotplus
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

importing and separating the data into features and target, we removed 'Sex' since this categorical data is binary which might not result in good desicion tree

In [11]:
train_df = pd.read_csv("data//train_heart.csv")

x_types = ['Age','ChestPainType','RestingBP','Cholesterol','FastingBS','RestingECG','MaxHR','ExerciseAngina','Oldpeak','ST_Slope']

x = train_df[x_types]
y = train_df[['HeartDisease']]

fixing our non-numeric values in the data

In [12]:
ce_ord = ce.OrdinalEncoder(cols = ['ChestPainType','RestingECG','ExerciseAngina','ST_Slope'])
x = ce_ord.fit_transform(x)

splitting the training data to test for the efficiency of the model

In [13]:
x_train, x_test, y_train, y_true = train_test_split(x, y, random_state=104, test_size=0.2)

creating and fitting the Decision tree model with our data. We set max depth to not overfit the model

In [14]:
model = GradientBoostingClassifier(n_estimators=1000, learning_rate=0.001, max_depth=4, random_state=42, min_impurity_decrease=1)
model = model.fit(x_train, y_train)

c:\Users\matej\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


visualization of our Decision Tree

In [15]:
#dot_data = StringIO()
#export_graphviz(model, out_file = dot_data, filled = True, rounded = True, special_characters = True, feature_names = x_types, class_names=['no','yes'])
#graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
#Image(graph.create_png(), width = 1500)

testing the model's efficiency by letting it predict on the unseen train data

In [16]:
y_test = model.predict(x_test)
y_true = np.array(y_true).flatten()
print(accuracy_score(y_true,y_test)*100, "%")

87.59689922480621 %


confusion metrix to visualize the efficiency

In [17]:
#confusion_matrix(y_true, y_test)
#ConfusionMatrixDisplay.from_predictions(y_true, y_test, labels=model.classes_)

making an actual prediction on the data for testing using our model

In [18]:
test_df = pd.read_csv("data//test_heart.csv")
test = test_df[x_types]
test = ce_ord.fit_transform(test)

prediction = model.predict(test)

out_df = pd.read_csv("data//sample_submission.csv")

for i,id in enumerate(out_df['id']):
    out_df['HeartDisease'][i] = prediction[i]

out_df.to_csv('output//outputGB.csv', encoding='utf-8', index=False)